<a href="https://colab.research.google.com/github/93model/Sec4_pro_Sentiment_analysis/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request

from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
train_moive_review_data = pd.read_table('ratings_train.txt')
test_moive_review_data = pd.read_table('ratings_test.txt')

In [ ]:
moive_review_data = pd.concat([train_moive_review_data, test_moive_review_data])
moive_review_data = moive_review_data[['document','label']]

print('전체 영화 리뷰 개수 :',len(moive_review_data)) 

In [ ]:
moive_review_data.head(5)

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")
shop_data = pd.read_table('ratings_total.txt', names=['ratings', 'document'])
print('전체  쇼핑 리뷰 개수 :',len(shop_data)) 

In [ ]:
shop_data['label'] = np.select([shop_data.ratings > 3], [1], default=0)
shop_data = shop_data[['document','label']]
shop_data.head(5)


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt", filename="sratings_total.txt")
steam_data = pd.read_table('ratings_total.txt', names=['label', 'document'])
steam_data = steam_data[['document','label']]
steam_data.head(5)


In [ ]:
print('전체 스팀 리뷰 개수 :',len(steam_data)) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
comu_val = pandas.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/감성대화말뭉치(원시데이터)_Validation.csv")
comu_train =  pandas.read_csv("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/감성대화말뭉치(최종데이터)_Training.csv")


In [ ]:
comu_data = pd.concat([comu_val,comu_train])
comu_data = comu_data[['감정_대분류','시스템응답1']]
print('전체 감정 응답 리뷰 개수 :',len(comu_data)) 

In [ ]:
comu_data['감정_대분류'].unique()

In [ ]:
comu_data.isnull().sum()

In [ ]:
comu_data['감정_대분류'].value_counts()


In [ ]:
change_value_dict = {'불안': 0, '기쁨': 1, '분노':0, '슬픔':0, '상처':0, '당황': None, '불안 ':0, '기쁨 ':1, 'nan' : None}
comu_data = comu_data.replace({'감정_대분류': change_value_dict})
comu_data['감정_대분류'].unique()

In [ ]:
comu_data.columns = ['label','document']
comu_data = comu_data[['document','label']]

In [ ]:
comu_data.isnull().sum()

In [ ]:
all_data = pd.concat([moive_review_data ,shop_data,steam_data,comu_data])
all_data.info()

전처리


In [ ]:
# 중복 제거
all_data.drop_duplicates(subset=['document'], inplace=True)
all_data.info()

In [ ]:
# 결측치 검사 
all_data.isnull().values.any()

In [ ]:
# 결측치 제거
all_data = all_data.dropna()
all_data.info()

In [ ]:
all_data.isnull().values.any()

학습 데이터 분리


In [ ]:
pip install sklearn

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(all_data, test_size=0.2)
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

In [ ]:
train_data['label'].value_counts().plot(kind = 'bar')
print(train_data.groupby('label').size().reset_index(name = 'count'))

In [ ]:
# 한글과 공백을 제외하고 모두 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how='any') # Null 값 제거
print('전처리 후 훈련용 샘플의 개수 :',len(train_data))

In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

In [ ]:
import pickle
 
## Save pickle

with open("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/train_data.pickle","wb") as fw:
    pickle.dump(train_data, fw)

    
with open("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/test_data.pickle","wb") as fw:
    pickle.dump(train_data, fw)

In [ ]:
# konlpy 사용
# 불용어 작성
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
okt = Okt()

In [ ]:
t_train = [] 
t_test = []
for sentence in train['document']:
    temp_X = okt.morphs(sentence, stem = True) 
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    t_train.append(temp_X)
    
for sentence in test['document']:
    temp_X = okt.morphs(sentence, stem = True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    t_test.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(t_train)
tokenizer.fit_on_texts(t_test)

In [ ]:
f_path = 'drive/MyDrive/'

In [ ]:
tokenizer

In [ ]:
import pickle
 
## Save pickle

with open("/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/all_data_tok.pickle","wb") as fw:
    pickle.dump(tokenizer, fw)

In [ ]:
!pip3 install pickle5

In [ ]:
threshold = 1
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt) * 100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq) * 100)

In [ ]:
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
X_train = tokenizer.texts_to_sequences(t_train)
X_test = tokenizer.texts_to_sequences(t_test)

In [ ]:
y_train = train['label']
y_test = test['label']

In [ ]:
print('문장의 최대 길이 :',max(len(l) for l in X_train))
print('문장의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))
 
 
max_len = 47
below_threshold_len(max_len,X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
from keras import models
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.5))
model.add(GRU(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='mse', metrics=['acc'])
history = model.fit(X_train, y_train , epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
loaded_model = load_model('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128, return_sequences = True))
model.add(Dense(1, activation='sigmoid'))

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/lstm_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='mse', metrics=['acc'])
history = model.fit(X_train, y_train , epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
lstm_model = load_model('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/lstm_model.h5')
print("\n 테스트 정확도: %.4f" % (lstm_model.evaluate(X_test, y_test)[1]))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/gru_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='mse', metrics=['acc'])
history = model.fit(X_train, y_train , epochs=5, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
gru_model = load_model('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/gru_model.h5')
print("\n 테스트 정확도: %.4f" % (gru_model.evaluate(X_test, y_test)[1]))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128, return_sequences = True))
model.add(GRU(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/lstm_gru_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='mse', metrics=['acc'])
history = model.fit(X_train, y_train , epochs=5, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
lstm_gru_model = load_model('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/lstm_gru_model.h5')
print("\n 테스트 정확도: %.4f" % (lstm_gru_model.evaluate(X_test, y_test)[1]))

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128, return_sequences = True))
model.add(GRU(128))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/lstm_gru_drop_out_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='mse', metrics=['acc'])
history = model.fit(X_train, y_train , epochs=5, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
lstm_gru_drop_out_model = load_model('/content/drive/MyDrive/Colab Notebooks/CS_AI_10/Sc4/Sc4_project/lstm_gru_drop_out_model.h5')
print("\n 테스트 정확도: %.4f" % (lstm_gru_drop_out_model.evaluate(X_test, y_test)[1]))

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 문장입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 문장입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('멍청이')

In [ ]:
sentiment_predict('사실 내가 런던에 8박 9일이라는 긴 일정을 잡은 이유는 뮤지컬 때문이다. 뉴욕에서 라이온 킹 단 한 편만 봤다는 게 너무나도 한이 되어서 런던에서 뮤지컬을 제대로 보기로 다짐했다. 그런데 매일 저녁 런던의 모든 극장에서 뮤지컬이 펼쳐지지만, 뮤지컬 좌석은 엄청 비싸다. 그나마 배우들의 얼굴이 보이 는 1층 자리는 적어도 최소 100파운드(15만 원)부터 시작하며 중 앙에 가깝고 앞으로 갈수록 비싸진다. 하지만, 나 같은 가난한 여 행객을 위해 평일 아침 일찍 극장에 줄을 서면, 약 10파운드 정도 의 가격에 맨 앞 좌석이나, 저렴한 좌석을 얻을 수 있는 데이 시트 (Day Seat) 라는 제도가 있다.')

In [ ]:
sentiment_predict('나는 오늘 데이트를 할 예정이라 기분이 좋다.')

In [ ]:
sentiment_predict('제작사 놈들 돈 많이 벌면서 게임 이따위로 만든다.')

In [ ]:
sentiment_predict('고기가 너무 딱딱하고 조금 탓어요')

In [ ]:
sentiment_predict('마동석은 멋있는데 영화는 재미가 없어요')

In [ ]:
sentiment_predict('내 남자한테 연락하지 마라')

In [ ]:
sentiment_predict('이번에 나온 시험은 망했다')

In [ ]:
# 트럼프 조사 예정 기사( https://www.fnnews.com/news/202206130918083308)
sentiment_predict('미 의사당 폭동에 대해 조사하고있는 하원 1.6조사위원회는 12일(현지시간) 역사상 유례가 없는 전직 대통령 도널드 트럼프에 대한 기소에 필요한 증거들이 충분히 발견되었다고 발표했다.AP통신과 미국 매체들에 따르면 하원 조사위원회는 2020년 대선 결과를 뒤집기 위해 트럼프 전 대통령이 사용한 갖가지 수법들이 법무부가 형사범으로 기소하기에 충분한 양이라고 밝혔다. 하원조사위는 트럼프의 선대본부장이었던 빌 스테피언도 13일 증인으로 소환되어 선거를 도둑 맞았다는 트럼프의 가짜 주장에 대해서 주로 진술하게 될 것이라고 말했다. 스테피언은 공개 청문회 증인으로 이미 소환장을 받았다. 9일(현지시간) 본격적으로 시작된 이번 청문회의 쟁점은 도널드 트럼프 전 대통령의 개입 여부이다. 특위는 의사당 폭동이 사실상 트럼프 전 대통령이 주도한 쿠데타 시도였다는 점을 부각하는데 주력했다.청문회 직전 조 바이든 대통령은 "명백한 헌법 위반"이라고 거세게 비판한 가운데, 트럼프 전 대통령은 "미국 역사상 가장 위대한 행동이었다"고 강조하며 정면 충돌했다. 뉴욕타임스(NYT) 등에 따르면 9일 오후 8시(한국시간 10일 오전 9시) 미 하원 1.6 조사 특별위원회의 공개 청문회가 황금시간대에 열리면서, CNN·NBC·ABC·CBS 등 대부분의 방송사가 특별 편성으로 이를 2시간여 동안 생중계했다.위원회는 앞으로 남은 청문회 일정에서도 트럼프와 측근들이 가짜 뉴스를 퍼뜨리고 법무부를 압박해 트럼프의 거짓 주장을 수용하도록 하고 심지어 당시 펜스 부통령에게까지 선거인단을 거부하게 만든 사실 등을 모두 폭로할 예정이다.트럼프 선대본부장이었던 스테피엔은 현재 와이오밍주 공화당 프라이머리의 트럼프가 추천하는 하원의원 후보 해리엣 해그먼의 캠프에서 선거총책을 맡고 있다. 트럼프 대변인 테일러 부도위치는 스테피언을 소환한 위원회의 결정은 순전히 정치적인 동기에서 나온 것이라고 비난했다.')